# Four components of a Chatbot
1. LLm
2. prompts
3. Memory
4. retriever

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os
os.environ["OPENAI_API_KEY"] = ""

chat = ChatOpenAI()
chat(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

AIMessage(content="J'adore programmer.")

In [2]:
# 一次发送多条消息
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(content="I love programming."),
]
chat(messages)

AIMessage(content="J'adore programmer.")

In [3]:
# 引入对话chain，这个chain自带记忆，并且自己就处理好记忆和当前的prompt之间的组合
from langchain.chains import ConversationChain
conversation = ConversationChain(llm = chat)
conversation.run("Translate this sentence from English to French: I love programming.")

'Je suis désolé, je ne peux pas traduire des phrases pour le moment.'

In [5]:
conversation.run("Translate this sentence from English to French: I love programming.")

"I'm sorry, I cannot translate sentences at the moment."

In [6]:
conversation.run("Translate it to German.")

"I'm sorry, I cannot translate sentences at the moment."

上面介绍的conversationchain自带memory。
但是我们可以自己手动指定memory类型，然后手动构建chain：

In [8]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder
)

llm = ChatOpenAI()
prompt = ChatPromptTemplate( # 这是关于message的template。 This is the template of Messages.
    messages=[
        SystemMessagePromptTemplate.from_template("You are a nice chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name='chat_history'),
        HumanMessagePromptTemplate.from_template('{question}')
    ]
)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

conversation.run({'question': "Hi"})





> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi

> Finished chain.


'Hello! How are you today?'

In [9]:
conversation(
    {"question": "Translate this sentence from English to French: I love programming."}
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi
AI: Hello! How are you today?
Human: Translate this sentence from English to French: I love programming.

> Finished chain.


{'question': 'Translate this sentence from English to French: I love programming.',
 'chat_history': [HumanMessage(content='Hi'),
  AIMessage(content='Hello! How are you today?'),
  HumanMessage(content='Translate this sentence from English to French: I love programming.'),
  AIMessage(content='Je suis désolé, je ne parle pas français. Cependant, la traduction de la phrase "I love programming" en français est "J\'adore programmer".')],
 'text': 'Je suis désolé, je ne parle pas français. Cependant, la traduction de la phrase "I love programming" en français est "J\'adore programmer".'}

In [10]:
conversation({"question": "Now translate the sentence to German."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi
AI: Hello! How are you today?
Human: Translate this sentence from English to French: I love programming.
AI: Je suis désolé, je ne parle pas français. Cependant, la traduction de la phrase "I love programming" en français est "J'adore programmer".
Human: Now translate the sentence to German.

> Finished chain.


{'question': 'Now translate the sentence to German.',
 'chat_history': [HumanMessage(content='Hi'),
  AIMessage(content='Hello! How are you today?'),
  HumanMessage(content='Translate this sentence from English to French: I love programming.'),
  AIMessage(content='Je suis désolé, je ne parle pas français. Cependant, la traduction de la phrase "I love programming" en français est "J\'adore programmer".'),
  HumanMessage(content='Now translate the sentence to German.'),
  AIMessage(content='Ich liebe das Programmieren.')],
 'text': 'Ich liebe das Programmieren.'}

下面和文档进行交谈

In [2]:
# 首先加载数据，并且存放到 vector store 中
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
'''
Split 之后返回的是 Document的列表。
每个Document都是一个字典。
每个Document的内容如下：
('page_content', "LLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)")
('metadata', {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'})
('type', 'Document')
其中的Metadata可以用于构建filters。
'''
print(len(all_splits))
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma # Chroma 是一个VS
vectorstore = Chroma.from_documents(documents= all_splits, embedding=OpenAIEmbeddings())


130


In [21]:
from langchain.memory import ConversationSummaryMemory
llm = ChatOpenAI()
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

retriever = vectorstore.as_retriever()
# 这里将retriever和memory和llm组合成chain。但是注意这里并没有使用prompt模板。所以就直接输入就行。
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)



In [18]:
qa("How do agents use Task decomposition?")

{'question': 'How do agents use Task decomposition?',
 'chat_history': [SystemMessage(content='')],
 'answer': 'Agents utilize task decomposition by breaking down large tasks into smaller, manageable subgoals. This allows them to handle complex tasks more efficiently. Task decomposition can be done using different approaches, such as using simple prompting with the help of Language Model LLM, task-specific instructions, or with human inputs. By decomposing tasks into subgoals, agents can plan ahead and prioritize their actions to achieve the overall objective.'}

总结：  
- 我们需要交流文档中的内容，因此需要构建文档retriever，也就需要构建vector store，然后还需要加载和预处理数据。
- 然后我们需要增加记忆模块。因为一般来说我们都是默认需要记忆的。
- 最后就是要将这三者结合起来，组成链。